In [1]:
# Assignment-2 of Pansetty Karthik (15110082)

In [2]:
import nltk
from matplotlib import pyplot as plt
import numpy as np
import scipy as sp
from urllib import request
import random
import re

from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
from sklearn.model_selection import train_test_split
import operator as op
from functools import reduce
import math
from itertools import islice

In [3]:
"""Question 1"""
# Importing the dataset

'Question 1'

In [4]:
url = "http://www.gutenberg.org/cache/epub/1661/pg1661.txt"

sherlock = request.urlopen(url)
sherlock_text = sherlock.read().decode('utf-8')

In [5]:
"""Question 2"""

'Question 2'

In [6]:
# Doing Text preprocessing by removing line breaks and punctuations

In [7]:
sherlock_text = re.sub('\n', ' ', sherlock_text)
sherlock_text = re.sub('\r', ' ', sherlock_text)
sherlock_text = re.sub('\ufeff', '', sherlock_text)

In [8]:
processed_text = []
for text in sherlock_text:
    punctuations = '''()-[]{};:"<>/?@#$%^&*_~'''
    no_punct = ""
    for char in text:
        if char not in punctuations:
            no_punct = no_punct + char
    processed_text.append(no_punct)
    
result= ''
for element in processed_text:
    result += str(element)
    
sherlock_text = result

In [9]:
from nltk.tokenize import sent_tokenize
sentences_list = sent_tokenize(sherlock_text)
len(sentences_list)

6198

In [10]:
# Splitting data into test and train

In [11]:
train_sent, test_sent = train_test_split(sentences_list,train_size=0.8)

/afs/crc.nd.edu/user/k/kpansett/anaconda3/envs/py36-test/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [12]:
# Adding <s> for start of sentence and </s> for end of a sentence

In [13]:
train_data = []
for i in train_sent:
    tokens = word_tokenize(i)
    train_data.append('<s>')
    train_data = train_data + tokens
    train_data.append('</s>')

In [14]:
test_data = []
for i in test_sent:
    tokens = word_tokenize(i)
    test_data.append('<s>')
    test_data = test_data + tokens
    test_data.append('</s>')

In [15]:
tokens = train_data

In [16]:
"""Question 3"""

'Question 3'

In [17]:
def Unigram(tokens):
    unigrams = ngrams(tokens, 1)
    unigram_list =tokens
    unigram_count_dict = dict(Counter(unigram_list))
    
    unigram_probability = {}
    for j in range(len(unigram_list)):
        unigram_probability[unigram_list[j]]  = unigram_count_dict[unigram_list[j]]/len(unigram_list)
    
    return unigram_list,unigram_count_dict,unigram_probability

In [18]:
"""Unigrams"""
unigram_list,unigram_count_dict,unigram_probability = Unigram(tokens)

In [19]:
vocab_size = len(unigram_count_dict)

In [20]:
# For n>1

def nGram(tokens,n):
    nGrams = ngrams(tokens, n)
    ngram_list =(list(nGrams))
    ngram_count_dict = dict(Counter(ngram_list))
    
    tGrams = ngrams(tokens, n-1)
    tgram_list =(list(tGrams))
    tgram_count_dict = dict(Counter(tgram_list))
    
    ngrams1 = list(ngram_count_dict.keys())
    ngram_probability={}
    ngram_probability_smoothing={}

    for j in range(len(ngrams1)):
        words = (ngrams1[j][0:n-1])
        ngram_probability[ngrams1[j]]  = ngram_count_dict[ngrams1[j]]/tgram_count_dict[words]
        ngram_probability_smoothing[ngrams1[j]]  = (ngram_count_dict[ngrams1[j]]+1)/(tgram_count_dict[words]+vocab_size)
    
    return ngram_list, ngram_count_dict, ngram_probability, ngram_probability_smoothing

In [21]:
"""Bigrams"""
bigram_list, bigram_count_dict, bigram_probability, bigram_probability_smoothing = nGram(tokens,2)

In [22]:
"""Trigrams"""
trigram_list, trigram_count_dict, trigram_probability, _ = nGram(tokens,3)

In [23]:
"""Quadgrams"""
quadgram_list, quadgram_count_dict, quadgram_probability, _ = nGram(tokens,4)

In [24]:
# To find number of nGrams possible
# Number of nGrams is nCr as the vocab consists of n words and we need r words in an r-gram 
# Number of Combinations = nCr

def ncr(n,r):
    ncr = math.factorial(n)/(math.factorial(n-r)*math.factorial(r))
    return int(ncr)

In [25]:
print('Vocab Size: ', vocab_size)
print()
print('no. of unigrams present: ', vocab_size)
print('no. of unigrams possible: ', vocab_size)
print()
print('no. of bigrams present: ', len(bigram_count_dict))
print('no. of bigrams possible: ', ncr(vocab_size,2))
print()
print('no. of trigrams present: ', len(trigram_count_dict))
print('no. of trigrams possible: ', ncr(vocab_size,3))
print()
print('no. of quadgrams present: ', len(quadgram_count_dict))
print('no. of quadgrams possible: ', ncr(vocab_size,4))

Vocab Size:  8518

no. of unigrams present:  8518
no. of unigrams possible:  8518

no. of bigrams present:  44134
no. of bigrams possible:  36273903

no. of trigrams present:  77204
no. of trigrams possible:  102969519316

no. of quadgrams present:  92316
no. of quadgrams possible:  219196364243935


In [26]:
"""Question 4"""

'Question 4'

In [27]:
def Generator(model_name):
    
    sentence=[]
    first_word = "<s>"
    sentence.append(first_word)
    max_length = 20
    
    if model_name == 'unigram':
        model = unigram_probability
        model_sorted=list(model.keys())
        current = sentence[0]
       
        while current != '</s>' and len(sentence)<max_length:
            random_word = random.choice(model_sorted)
            sentence.append(random_word)
            current = random_word
        
        complete_sentence = ' '.join(sentence)
        return complete_sentence 
        
    elif model_name == 'bigram':
        model = bigram_probability
        model_sorted=list(model.keys())
        current = sentence[0]

        
        while current != '</s>' and len(sentence)<max_length:
            if current == "<s>" and len(sentence)<2:
                random_word = random.choice(list(unigram_probability.keys()))
                sentence.append(random_word)
                current = random_word
            else:
                tmp_key=[]
                tmp_val=[]
                for i in range(len(model_sorted)):
                    if model_sorted[i][0] == current:
                        tmp_key.append(model_sorted[i][1])
                        tmp_val.append(model[model_sorted[i]])

                index, value = max(enumerate(tmp_val), key=op.itemgetter(1))
                next_word = tmp_key[index]
                sentence.append(next_word)
                current = next_word
            
        complete_sentence = ' '.join(sentence)
        return complete_sentence 
    
    
    elif model_name == 'trigram':
        model = trigram_probability
        model_sorted=list(model.keys())
        current = sentence[0]

        
        while current != '</s>' and len(sentence)<max_length:
            if current == "<s>" and len(sentence)<3:
                random_word = random.choice(list(bigram_probability.keys()))
                sentence.append(random_word[0])
                sentence.append(random_word[1])
                current = random_word[1]
            else:
                tmp_key=[]
                tmp_val=[]
                for i in range(len(model_sorted)):
                    if model_sorted[i][0] == sentence[-2] and model_sorted[i][1] == sentence[-1]:
                        tmp_key.append(model_sorted[i][2])
                        tmp_val.append(model[model_sorted[i]])

                index, value = max(enumerate(tmp_val), key=op.itemgetter(1))
                next_word = tmp_key[index]
                sentence.append(next_word)
                current = next_word
            
        complete_sentence = ' '.join(sentence)
        return complete_sentence  
    
    elif model_name == 'quadgram':
        model = quadgram_probability
        model_sorted=list(model.keys())
        current = sentence[0]

        
        while current != '</s>' and len(sentence)<max_length:
            if current == "<s>" and len(sentence)<4:
                random_word = random.choice(list(trigram_probability.keys()))
                sentence.append(random_word[0])
                sentence.append(random_word[1])
                sentence.append(random_word[2])
                current = random_word[2]
            else:
                tmp_key=[]
                tmp_val=[]
                for i in range(len(model_sorted)):
                    if model_sorted[i][0] == sentence[-3] and model_sorted[i][1] == sentence[-2] and model_sorted[i][2] == sentence[-1]:
                        tmp_key.append(model_sorted[i][3])
                        tmp_val.append(model[model_sorted[i]])

                index, value = max(enumerate(tmp_val), key=op.itemgetter(1))
                next_word = tmp_key[index]
                sentence.append(next_word)
                current = next_word
            
        complete_sentence = ' '.join(sentence)
        return complete_sentence  

In [28]:
Generator('unigram')

'<s> shone plugs lodginghouse glance severely regret anxiously outrages point lenses encamp common prices passage shading ha purchasing partly Arms'

In [29]:
Generator('bigram')

'<s> wellopened eye . </s>'

In [30]:
Generator('trigram')

"<s> blue stone of the house , and I have no doubt that the doctor 's advice and help ,"

In [31]:
Generator('quadgram')

'<s> will have informed the police of Savannah that these three gentlemen are badly wanted here upon a charge of'

In [32]:
# 1 
# -
# 2
# log_probability = unigram_probability[sentence_tokens[0]]
# 3
# log_probability = unigram_probability[sentence_tokens[0]] + bigram_probability[(sentence_tokens[0],sentence_tokens[1])]
# 4
# log_probability = unigram_probability[sentence_tokens[0]] + bigram_probability[(sentence_tokens[0],sentence_tokens[1])] + trigram_probability[(sentence_tokens[0],sentence_tokens[1], sentence_tokens[2])]

In [33]:
def Probability(sentence,model_name):
    
    log_probability = 0
    sentence_tokens = sentence.split()
    
    if model_name == 'unigram':
        for i in range(len(sentence_tokens)):
            log_probability = log_probability + math.log2(unigram_probability[sentence_tokens[i]])
    
    elif model_name == 'bigram':
        for i in range(1,len(sentence_tokens)):
            log_probability = log_probability + math.log2(bigram_probability[(sentence_tokens[i-1],sentence_tokens[i])])
            
    elif model_name == 'trigram':
        for i in range(2,len(sentence_tokens)):
            log_probability = log_probability + math.log2(trigram_probability[(sentence_tokens[i-2],sentence_tokens[i-1],sentence_tokens[i])])
    
    elif model_name == 'quadgram':
        for i in range(3,len(sentence_tokens)):
            log_probability = log_probability + math.log2(quadgram_probability[(sentence_tokens[i-3],sentence_tokens[i-2],sentence_tokens[i-1],sentence_tokens[i])])
    
    return log_probability
    

In [35]:
Probability('blue stone of the house','unigram')

-45.53640820473585

In [36]:
Probability('blue stone of the house','bigram')

-16.963722837938732

In [37]:
"""Question 5"""

'Question 5'

In [38]:
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [ ]:
# Printing first ten probabilities of bigrams and add-1 smoothed bigrams

In [39]:
take(10,bigram_probability_smoothing.items())

[(('<s>', 'The'), 0.018477292965271595),
 (('The', 'inspector'), 0.00034110289937464467),
 (('inspector', 'and'), 0.0005858917272088118),
 (('and', 'two'), 0.0009271277582050807),
 (('two', 'men'), 0.0005791729410401946),
 (('men', 'accompanied'), 0.0002338907730090048),
 (('accompanied', 'her'), 0.00023476933912431035),
 (('her', 'back'), 0.0003391746749576032),
 (('back', ','), 0.001159151501101194),
 ((',', 'and'), 0.0743481803367735)]

In [40]:
take(10, bigram_probability.items())

[(('<s>', 'The'), 0.0500201694231545),
 (('The', 'inspector'), 0.007220216606498195),
 (('inspector', 'and'), 0.25),
 (('and', 'two'), 0.003968253968253968),
 (('two', 'men'), 0.034782608695652174),
 (('men', 'accompanied'), 0.030303030303030304),
 (('accompanied', 'her'), 1.0),
 (('her', 'back'), 0.0061162079510703364),
 (('back', ','), 0.08256880733944955),
 ((',', 'and'), 0.17616747181964573)]

In [62]:
# 

In [66]:
"""If we observe in the above values ('inspector', 'and') and ('accompanied', 'her') have drastic change in probabilities. We see this because there is a change of count in the bigrams when we add smoothing. For example if there is a bigram occuring only one time (whose both words also occur only in this bigram), its probability will be decreased drastically when we smoothen it"""

"If we observe in the above values ('inspector', 'and') and ('accompanied', 'her') have drastic change in probabilities. We see this because there is a change of count in the bigrams when we add smoothing. For example if there is a bigram occuring only one time (whose both words also occur only in this bigram), its probability will be decreased drastically when we smoothen it"

In [41]:
"""Question 6"""

'Question 6'

In [42]:
def GoodTuring(bigram_count_dict):
    b = bigram_count_dict.values()

    frequencies = {}

    for i in b:
        if i not in frequencies:
            frequencies[i] = 1
        else:
            frequencies[i]+=1

    c_star = {}
    for i in range(1,10):
        c_star[i] = ((i+1)*frequencies[i+1])/frequencies[i]

    return c_star

In [43]:
good_turing = GoodTuring(bigram_count_dict)

In [44]:
good_turing

{1: 0.3225201801856275,
 2: 1.109111361079865,
 3: 2.026369168356998,
 4: 2.932932932932933,
 5: 3.7781569965870307,
 6: 5.1409214092140925,
 7: 6.878228782287823,
 8: 6.334763948497854,
 9: 9.329268292682928}

In [45]:
d_val = {}

for key, val in good_turing.items():
    d_val[key] = key-val

In [46]:
d_val

{1: 0.6774798198143726,
 2: 0.8908886389201349,
 3: 0.973630831643002,
 4: 1.0670670670670672,
 5: 1.2218430034129693,
 6: 0.8590785907859075,
 7: 0.12177121771217703,
 8: 1.6652360515021458,
 9: -0.32926829268292757}

In [47]:
d_val_obtained = sum(list(d_val.values()))/5

In [48]:
d_val_obtained

1.4295453856349698

In [49]:
good_turing_prob = bigram_probability

for key in bigram_count_dict:
    if bigram_count_dict[key] in good_turing:
        good_turing_prob[key] = good_turing[bigram_count_dict[key]]/unigram_count_dict[key[0]]

In [51]:
"""Question 7"""

'Question 7'

In [52]:
# For Test dataset

In [53]:
bigram_list_test = list(ngrams(test_data,2))

In [54]:
# Finding Perplexity for Add-One Smoothing and Good Turing Smoothing

In [55]:
num = len(test_data)

In [56]:
def bigram_perplexity(bigram_list_test, bigram_probability):
    
    prod_bigrams = 1
    for i in bigram_list_test:
        if i in bigram_probability:
            prod_bigrams = prod_bigrams*(np.power(1/bigram_probability[i],1/num))
    
        else:
            prod_bigrams = prod_bigrams*(np.power(vocab_size,1/num))
    perplexity = (prod_bigrams)
    
    return perplexity

In [57]:
perplexity_add1 = bigram_perplexity(bigram_list_test,bigram_probability_smoothing)

In [58]:
perplexity_good_turing = bigram_perplexity(bigram_list_test, good_turing_prob)

In [59]:
perplexity_add1

863.5598688263821

In [60]:
perplexity_good_turing

151.66786744662392